<a href="https://colab.research.google.com/github/wangqianyang/colab/blob/main/rvecolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 文件设置
INPUT_FILE = "input.mp4" # 输入视频文件名称

# 这是将导出到 Google Drive “我的云端硬盘” 下的 “RVE” 文件夹中的输出文件。将覆盖任何现有文件。
OUTPUT_FILE = "output.mkv" # 推荐使用 mkv 格式，因为它支持更多编解码器 {mkv, mp4, avi, mov}

KILL_RUNTIME_AFTER_RENDER = True # 渲染完成后终止运行时，以节省 Colab 计算资源。如果复制到云端硬盘失败，需要手动下载文件。

# 插帧设置

# https://github.com/TNTwise/real-video-enhancer-models/releases/tag/models/ 是 RVE 所有模型文件的存储位置
# 您可以从那里下载，或者尝试 https://openmodeldb.info/ 中的自定义模型
INTERPOLATE = True # {True, False} 是否启用插帧
INTERPOLATE_FACTOR = 2 # 插帧倍数
SCENE_DETECT_METHOD = "pyscenedetect" # {pyscenedetect, none} 场景检测方法
SCENE_DETECT_SENSATIVITY = "3.5" # {0 - 9.9} 场景检测敏感度，值越低越敏感

"""
内部插帧模型:
rife4.6.pkl - 速度优先
rife4.7.pkl - 平滑度优先
rife4.18.pkl - 真实感
rife4.22.pkl - 动画
rife4.25.pkl - 通用
GMFSS.pkl - 动画
GMFSS_PRO.pkl - 动画
GIMMVFI_RAFT.pth - 真实感
"""
INTERPOLATE_MODEL = "rife4.25.pkl" # {内部模型名称, (模型路径链接)}


# 超分设置
UPSCALE = True # {True, False} 是否启用超分
"""
4xNomos8k_span_otf_weak.pth - 真实感 4x 高质量输入
4xNomos8k_span_otf_medium.pth - 真实感 4x 中质量输入
4xNomos8k_span_otf_strong.pth - 真实感 4x 低质量输入
2x_BHI_SpanPlusDynamic_Light.pth - 真实感 2x 高质量输入 (仅限 pytorch, 实验性)

2x_ModernSpanimationV2.pth - 动画 2x
2x_ModernSpanimationV3.pth - 动画 2x (仅限 pytorch, 实验性)
2x_AnimeJaNai_V2_Compact_36k.pth - 动画 2x
2x_AnimeJaNai_HD_V3_Sharp1_Compact_430k.pth - 动画 2x
up2x-latest-conservative.pth - 动画 2x (仅限 pytorch, 慢)
up3x-latest-conservative.pth - 动画 3x (仅限 pytorch, 慢)
up4x-latest-conservative.pth - 动画 4x (仅限 pytorch, 慢)
"""
UPSCALE_MODEL = "2x_ModernSpanimationV2.pth" # {(模型路径链接)}

# FFMpeg 设置
VIDEO_ENCODER = "libx264" # {libx264,libx265,vp9,av1,prores,ffv1,x264_vulkan,x264_nvenc,x265_nvenc} 视频编码器
AUDIO_ENCODER = "copy_audio" # {aac,libmp3lame,opus,copy_audio} 音频编码器
SUBTITLE_ENCODER = "copy_subtitle" # {srt,ass,webvtt,copy_subtitle} 字幕编码器

# 后端设置
BACKEND = "pytorch" # {pytorch, tensorrt} TensorRT 最快，但安装依赖项需要很长时间。

In [ ]:
# 渲染完成后将输出文件移动到云端硬盘以进行持久存储
import os
import subprocess
import shutil
from google.colab import drive, output

# 挂载云端硬盘并将输出文件移动到那里以进行持久存储
drive.mount('/content/drive')

In [ ]:
# 删除现有的 real-video-enhancer 目录，以确保干净克隆
!rm -rf real-video-enhancer/
# 从 GitHub 克隆 real-video-enhancer 仓库的 2.3.8 分支
!git clone https://github.com/tntwise/real-video-enhancer --branch 2.3.8

In [ ]:
import os
# 创建用于存放模型的目录
!mkdir real-video-enhancer/models/
# 记录当前工作目录
origdir = os.getcwd()
# 更改工作目录到模型文件夹
os.chdir("real-video-enhancer/models/")

# 检查是否指定了模型
def has_model(model: str):
    return model and model.lower() != "none"

# 下载指定模型文件
def download_model(model: str):
    os.system(f"wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/{model}")


# 通过提取链接末尾来下载外部模型
def download_external_model(model_link : str):
        print("Downloading Model")
        os.system(f"wget {model_link}")
        model = model_link.split(r'/')[-1]
        return model


# 如果启用了插帧功能，则下载插帧模型
if INTERPOLATE:
    if "http" in INTERPOLATE_MODEL: # 如果模型路径是 URL
        UPSCALE_MODEL = download_external_model(INTERPOLATE_MODEL)
    else: # 否则从预定义源下载内部模型
        download_model(INTERPOLATE_MODEL)

# 如果启用了超分功能，则下载超分模型
if UPSCALE:
    if "http" in UPSCALE_MODEL:
        print("up")
        UPSCALE_MODEL = download_external_model(UPSCALE_MODEL)
    else:
        download_model(UPSCALE_MODEL)

# 恢复原始工作目录
os.chdir(origdir)

In [ ]:
# 创建 backend 目录，防止 RVE 崩溃。这是为了防止 RVE 未检测到 "backend" 文件夹，导致无法写入不存在的日志文件位置。
!mkdir backend

In [ ]:
if BACKEND.lower() == 'tensorrt':
    # 下载 Python 独立构建，用于 TensorRT 后端
    !wget https://github.com/astral-sh/python-build-standalone/releases/download/20250317/cpython-3.12.9+20250317-x86_64-unknown-linux-gnu-install_only.tar.gz
    # 解压 Python 构建
    !tar xvf cpython-3.12.9+20250317-x86_64-unknown-linux-gnu-install_only.tar.gz
    # 如果使用 TensorRT，则使用独立 Python 安装所需的依赖项
    !./python/bin/python3 -m pip install  --extra-index-url https://download.pytorch.org/whl/cu126 -r real-video-enhancer/backend/requirements.txt # 仅在使用 tensorrt 时安装

# 创建 bin 目录并下载 ffmpeg 可执行文件，然后赋予执行权限
!mkdir bin/ && cd bin && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/ffmpeg && chmod +x ffmpeg

In [ ]:
# 示例超分/降噪命令
# 建议使用 --video_encoder_preset x264_nvenc
def build_backend_command() -> str:
    command = (('python3 real-video-enhancer/backend/rve-backend.py' if BACKEND == "pytorch" else './python/bin/python3 real-video-enhancer/backend/rve-backend.py')
               + f' -i "{INPUT_FILE}"'
               + f' -o "{OUTPUT_FILE}"'
               + f' -b {BACKEND} '
               + f' --video_encoder_preset {VIDEO_ENCODER}'
               + f' --audio_encoder_preset {AUDIO_ENCODER}'
               + f' --subtitle_encoder_preset {SUBTITLE_ENCODER}')

    # 如果启用了超分，则添加超分模型参数
    if UPSCALE_MODEL and UPSCALE:
        command += f" --upscale_model real-video-enhancer/models/{UPSCALE_MODEL} "

    # 如果启用了插帧，则添加插帧模型及相关参数
    if INTERPOLATE_MODEL and INTERPOLATE:
        command += f" --interpolate_model real-video-enhancer/models/{INTERPOLATE_MODEL} "
        command += f" --interpolate_factor {INTERPOLATE_FACTOR} "
        command += f" --scene_detect_method {SCENE_DETECT_METHOD} "
        command += f" --scene_detect_threshold {SCENE_DETECT_SENSATIVITY} "

    return command

# 将构建的命令写入 command.sh 文件
with open("command.sh", "w") as f:
  f.write(build_backend_command())

# 执行 command.sh 脚本来运行 RVE 后端处理
!bash command.sh

Using backend: pytorch
Scene Detection Enabled
Using Device: Tesla T4
Using Interpolation Model: real-video-enhancer/models/rife4.25.pkl


FPS: 6 Current Frame: 968 ETA: 0:08:58Exception ignored in: <module 'threading' from '/usr/lib/python3.11/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1590, in _shutdown
    lock.acquire()
KeyboardInterrupt: 
terminate called without an active exception
command.sh: line 1:  3025 Aborted                 (core dumped) python3 real-video-enhancer/backend/rve-backend.py -i "https://v.animethemes.moe/OshiNoKo-OP1-NCBD1080.webm" -o "output.mkv" -b pytorch --video_encoder_preset libx264 --audio_encoder_preset copy_audio --subtitle_encoder_preset copy_subtitle --interpolate_model real-video-enhancer/models/rife4.25.pkl --interpolate_factor 2 --scene_detect_method pyscenedetect --scene_detect_threshold 3.5


In [ ]:
# Colab 中的源文件路径
source_path = f'/content/{OUTPUT_FILE}'

# Google Drive (根目录) 上的目标路径
destination_path = f'/content/drive/My Drive/RVE/{OUTPUT_FILE}'

# 如果目标文件夹不存在，则创建它
if not os.path.exists(os.path.dirname(destination_path)):
  os.mkdir(os.path.dirname(destination_path))

# 移动文件
try:
  shutil.copy(source_path, destination_path)
except Exception as e:
  print(f"复制到云端硬盘失败！ {e}\n请在终止运行时之前下载所有已正确渲染的文件！")
  input("按 Enter 终止运行时。")

  # 终止运行时
  output.eval_js('google.colab.kernel.disconnect()')
  os.kill(os.getpid(), 9)

# 如果设置了渲染后终止运行时，则执行终止操作
if KILL_RUNTIME_AFTER_RENDER:

  # 终止运行时
  output.eval_js('google.colab.kernel.disconnect()')
  os.kill(os.getpid(), 9)